In [1]:
import torch
import torch.nn as nn
from torchvision import models
from train import read_data

In [2]:
class Features(nn.Module):
    def __init__(self, arch):
        super(Features, self).__init__()
        
        if arch == 'resnet152':
            model = models.resnet152(pretrained=True)
            self.features = nn.Sequential(*list(model.children())[:-1])
        elif arch == 'densenet169':
            model = models.densenet169(pretrained=True)
            self.features = nn.Sequential(*list(model.children())[:-1])
            self.features.add_module('relu_e', nn.ReLU(inplace=True))
            self.features.add_module('avg_pool_e', nn.AdaptiveAvgPool2d((1, 1)))
        elif arch == 'inception_v3':
            model = models.inception_v3(pretrained=True)
            self.features = nn.Sequential(*list(model.children())[:-1])
            self.features._modules.pop('13')
            self.features.add_module('avg pool', nn.AvgPool2d(35))
        else:
            raise Exception("arch error")
            
        
        for param in self.features.parameters():
            param.requires_grad = False
            
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x
            

In [3]:
train_features = {
    'resnet152': [],
    'densenet169': [],
    'inception_v3': []    
}
valid_features = {
    'resnet152': [],
    'densenet169': [],
    'inception_v3': []    
}
test_features = {
    'resnet152': [],
    'densenet169': [],
    'inception_v3': []    
}

In [4]:
split_p = 0.2
batch_size = 64
labels_csv = './labels.csv'

In [ ]:
root = 'D:/DATA/dog breed'
train_data = read_data(root + '/train', 'train', labels_csv, split_p=split_p)
valid_data = read_data(root + '/train', 'valid', labels_csv, split_p=split_p)
test_data = read_data(root + '/test', 'test', labels_csv, split_p=0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size)

In [ ]:
def get_features_labels(arch, data_loader, feature_list):
    print('{} begining...'.format(arch))
    model = Features(arch)
    if torch.cuda.is_available():
        model.cuda()
        torch.backends.cudnn.benchmark = True
    with torch.no_grad():
        for x, y in data_loader:
            if torch.cuda.is_available():
                x = x.cuda()
                z = model(x)
                feature_list.extend((f, l) for f, l in zip(z.cpu(), y))
    torch.cuda.empty_cache()
    print('{} done...'.format(arch))

In [ ]:
def get_features(arch, test_loader, feature_list):
    print('{} begining...'.format(arch))
    model = Features(arch)
    if torch.cuda.is_available():
        model.cuda()
        torch.backends.cudnn.benchmark = True
    with torch.no_grad():
        for x in test_loader:
            if torch.cuda.is_available():
                x = x.cuda()
                z = model(x)
                feature_list.extend(z.cpu())
    torch.cuda.empty_cache()
    print('{} done...'.format(arch))

In [ ]:
for arch, feature_list in train_features.items():
    if arch != 'inception_v3':
        get_features_labels(arch, train_dataloader, feature_list)

resnet152 begining...
resnet152 done...
densenet169 begining...


C:\Users\ANG\AppData\Local\conda\conda\envs\ml\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


densenet169 done...


In [ ]:
for arch, feature_list in valid_features.items():
    if arch != 'inception_v3':
        get_features_labels(arch, valid_dataloader, feature_list)

resnet152 begining...
resnet152 done...
densenet169 begining...
densenet169 done...


In [ ]:
for arch, feature_list in test_features.items():
    if arch != 'inception_v3':
        get_features(arch, test_dataloader, feature_list)

resnet152 begining...
resnet152 done...
densenet169 begining...
densenet169 done...


In [ ]:
root = 'D:/DATA/dog breed'
train_data = read_data(root + '/train', 'train', labels_csv, split_p=split_p, mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
valid_data = read_data(root + '/train', 'valid', labels_csv, split_p=split_p, mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
test_data = read_data(root + '/test', 'test', labels_csv, split_p=0, mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])

train_dataloader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size)

get_features_labels('inception_v3', train_dataloader, train_features['inception_v3'])
get_features_labels('inception_v3', valid_dataloader, valid_features['inception_v3'])
get_features('inception_v3', test_dataloader, test_features['inception_v3'])

inception_v3 begining...
inception_v3 done...
inception_v3 begining...
inception_v3 done...
inception_v3 begining...


In [ ]:
torch.save(train_features, root + '/train_features.pth')
torch.save(valid_features, root + '/valid_features.pth')
torch.save(test_features, root + '/test_features.pth')